In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000000)
pd.options.mode.chained_assignment = None 

In [3]:
clean_df = pd.read_csv('data/play_by_play_2016.csv').drop(columns='Unnamed: 0')

In [4]:
remove_cols = ['Fielders Choice', 'Sacrifice Fly', 'Fielder\'s Choice-Adv-2nd', 
               'Sacrifice Bunt', 'Reached on error-Adv-3rd', 'Reached on error-Adv-2nd', 
               'Catcher Interference', 'Sacrifice Bunt-Adv-2nd', 'Sacrifice Bunt', 
               'Dirt Ball', 'Sacrifice Fly-Adv-2nd', 'Fielders Choice - Out at 2nd', 
               'Sacrifice Fly-Adv-1st', 'Sacrifice Bunt-Adv-1st', 'Reached On Error - Out at 2nd', 
               'Balk', 'Reached on error', 'Hit by pitch']

high_level_groupings = {
    'ball' : ['Ball', 'Intentional Ball', 'Pitch Out'], 
    
    'strike' : 
        ['Strike Looking', 'Strike Swinging', ' Strike Swinging - Out at Home', 
         'Strike-swinging-Adv-1st', 'Strike-looking-Adv-1st', 'Single - Out at 3rd', 
         'Strike Looking - Out at 1st'], 
    
    'contact_in_play_out' : 
        ['Pop Out', 'Ground Out', 'Fly Out', 'Line Out'], 
    
    'contact_in_on_base' : 
        ['Homerun', 'Single', 'Double', 'Single - Out at 2nd', 'Triple', 'Single-Adv-2nd', 
         'Double-Adv-3rd', 'Double Out at 3rd', 'Single-Adv-Home', 'Single-Adv-3rd', 'Double-Adv-Home', 
         'Triple-Adv-Home', 'Triple - Out at Home', 'Double-Adv-Home'], 
    
    'contact_foul' : ['Foul Ball', 'Foul Tip', ]
}

In [5]:
# remove any bat_id with 'rare' events (~10% of at-bats)
remove_ids = clean_df.loc[clean_df['outcomeDescription'].isin(remove_cols)]['bat_id'].unique()
clean_df = clean_df.loc[~clean_df['bat_id'].isin(remove_ids)]

# theres currently no assignment for a walk; create new column for outcome called walk
clean_df['walk_flag'] = np.where(
    ((clean_df['startingBalls'] == 3) & 
     (clean_df['outcomeDescription'].isin(high_level_groupings['ball']))), 1, 0
)

# do the same for strikes
clean_df['strikeout_flag'] = np.where(
    ((clean_df['startingStrikes'] == 2) & 
    (clean_df['outcomeDescription'].isin(high_level_groupings['strike']))), 1, 0
)

# put the previous ids in the data
clean_df['prev_state_id'] = (
    clean_df.sort_values(by=['bat_id', 'event_num'], ascending=True)
      .groupby('bat_id')['outcomeId']
      .shift(1)
      .fillna('bFP')
)

clean_df['prev_state_desc'] = (
    clean_df.sort_values(by=['bat_id', 'event_num'], ascending=True)
      .groupby('bat_id')['outcomeDescription']
      .shift(1)
      .fillna('First Pitch')
)

# assign based on the dictonary above
clean_df['assigned_outcome'] = ''
clean_df['prev_assigned_outcome'] = ''
for outcome, grouping in high_level_groupings.items(): 
    clean_df['assigned_outcome'] = np.where(
        clean_df['walk_flag'] == 1, 'walk', 
            np.where(clean_df['strikeout_flag'] == 1, 'strikeout',
                np.where(
                    clean_df['outcomeDescription'].isin(grouping), outcome, clean_df['assigned_outcome']
                )
            )          
        )
    
    # has the previous outcomes as the same grouping (except itis imporssible to get a walk here)
    clean_df['prev_assigned_outcome'] = np.where(
            clean_df['prev_state_desc'] == 'First Pitch', 'first_pitch',
            np.where(
                clean_df['prev_state_desc'].isin(grouping), outcome, clean_df['prev_assigned_outcome']
            )
    )

There are some fluke occurrences that will lead to werid things in the data. I'll remove these for this analysis rather than trying to compensate (about 450 AB's)

For an example (Home Run called back on second pitch), see here: https://www.mlb.com/video/search?q=BatterId+%3D+%5B467793%5D+AND+PlayerId+%3D%3D+%5B112526%5D+AND+Inning+%3D+%5B1%5D+Order+By+Timestamp+DESC



In [6]:
bad_cols = ['contact_in_on_base', 'contact_in_play_out']
bad_records = clean_df.loc[clean_df['prev_assigned_outcome'].isin(bad_cols)]['bat_id'].unique()

# remove these records from the data
clean_df = clean_df.loc[~clean_df['bat_id'].isin(bad_records)]

### Get the transition matrix for the Entire MLB

In [7]:
def compute_transition_matrix(matrix_df): 
    transition_matrix = pd.pivot_table(
        data=matrix_df[['pitch_id', 'assigned_outcome', 'prev_assigned_outcome']], 
        index='prev_assigned_outcome', 
        columns='assigned_outcome',
        aggfunc='count', 
        fill_value=0
    )

    transition_matrix.columns = [i[1] for i in transition_matrix.columns.to_flat_index()]
    transition_matrix = (
        transition_matrix.reset_index().rename(columns={'prev_assigned_outcome' : 'previous_play'})
    )

    transition_matrix['total'] = transition_matrix.sum(axis=1)
    for col in [i for i in transition_matrix.columns if i not in ['previous_play', 'total']]: 
        transition_matrix[col] = transition_matrix[col] / transition_matrix['total']

    # transition_matrix = transition_matrix.drop(columns='total')
    return transition_matrix

In [8]:
# see the matrix for the entire mlb
compute_transition_matrix(matrix_df=clean_df)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  from ipykernel import kernelapp as app


,previous_play,ball,contact_foul,contact_in_on_base,contact_in_play_out,strike,strikeout,walk,total
0,ball,0.278140,0.199762,0.073851,0.136458,0.224339,0.048297,0.039153,192374
1,contact_foul,0.320865,0.234311,0.077727,0.152527,0.062540,0.129723,0.022307,102526
2,first_pitch,0.386704,0.107284,0.041843,0.073729,0.390428,0.000007,0.000007,152763
3,strike,0.361585,0.198030,0.066151,0.132555,0.160950,0.068333,0.012396,130127


### Get this for a certain batter and at a certain time (pitch count)

### Compute the transition matrix at every point in a certain at bat

In [9]:
def prob_next_state(hitter_matrix, pitcher_matrix): 
    
    # get the columns that are in pitcher but not in hitter (for 0 prob)
    h_missing_cols = [i for i in pitcher_matrix.columns if i not in hitter_matrix.columns]
    p_missing_cols = [i for i in hitter_matrix.columns if i not in pitcher_matrix.columns]

    # add the columns with 0's if not in
    if len(h_missing_cols) > 0:
        for i in h_missing_cols: 
            hitter_matrix[i] = 0

    if len(p_missing_cols) > 0:
        for i in p_missing_cols: 
            pitcher_matrix[i] = 0

    # make sure the rows are (for prev play) are in the same order
    common_rows = [
        i for i in hitter_matrix['previous_play'].unique() 
        if i in pitcher_matrix['previous_play'].unique()
    ]

    # get the common order between the two
    row_order = {i:j for i,j in enumerate(common_rows)}
    
    # force the dataframes to be in that order
    hitter_m = hitter_matrix.sort_values(by=['previous_play'], key=lambda x: x.map(row_order))
    pitcher_m  = pitcher_matrix.sort_values(by=['previous_play'], key=lambda x: x.map(row_order))
    
    results_df = pd.DataFrame()
    cols = [i for i in hitter_matrix.columns if i not in ['previous_play', 'total']]
    results_df['previous_play'] = common_rows
    for col in cols:
        results_df[col] = cumulative_prob(
            pa=hitter_matrix[col], 
            pb=pitcher_matrix[col], 
            an=hitter_matrix['total'], 
            bn=pitcher_matrix['total']
    )
        
    return results_df

def compute_hitter_matrix(clean_df, hitter, strikes, balls, bat_id): 
    # compute for all the hitter matrix
    hitter_state_df = clean_df.loc[
        ((clean_df['hitter_full_name'] == hitter) & 
         (clean_df['bat_id'] != bat_id) & 
         (clean_df['startingStrikes'] == strikes) & 
         (clean_df['startingBalls'] == balls))
    ]
    
    hitter_matrix = compute_transition_matrix(matrix_df=hitter_state_df)
    return hitter_matrix

def compute_pitcher_matrix(clean_df, pitcher, strikes, balls, bat_id): 
    pitcher_state_df = clean_df.loc[
        ((clean_df['pitcher_full_name'] == pitcher) & 
         (clean_df['bat_id'] != bat_id) & 
         (clean_df['startingStrikes'] == strikes) & 
         (clean_df['startingBalls'] == balls))
    ]
    
    pitcher_matrix = compute_transition_matrix(matrix_df=pitcher_state_df)
    return pitcher_matrix

def cumulative_prob(pa, pb, an, bn): 
    a_prob = pa * (1-bn/(an+bn))
    b_prob = pb * (1-an/(bn+an))
    cumul_prob = a_prob + b_prob
    return cumul_prob

In [37]:
# choose 20000 random at-bats
r_bat_id = random.randint(0, len(clean_df['bat_id'].unique()))
n = 20000
test_bat_ids = clean_df['bat_id'].unique()[r_bat_id:r_bat_id+n]

In [38]:
results_df = pd.DataFrame()
for counter_i, test_bat_id in enumerate(test_bat_ids): 
    bat_df = clean_df.loc[clean_df['bat_id'] == test_bat_id]
    
    # get the name of the hitter and pitcher
    hitter, pitcher = bat_df['hitter_full_name'].unique().item(), bat_df['pitcher_full_name'].unique().item()
    total_events = sorted(bat_df['event_num'].tolist())

    for event in total_events: 
        # get the count at that event in time
        count_df = bat_df.loc[bat_df['event_num'] == event]
        pitch_id = count_df['pitch_id'].item()
        prev_outcome = count_df['prev_assigned_outcome'].item()
        actual_outcome = count_df['assigned_outcome'].item()
        balls, strikes = count_df['startingBalls'].item(), count_df['startingStrikes'].item()
        
        # pull the transition matrix for all times this count occured in the dataset
        # remove this certain at bat for both hitter and pitcher
        hitter_matrix = compute_hitter_matrix(
            clean_df=clean_df, 
            hitter=hitter, 
            strikes=strikes, 
            balls=balls, 
            bat_id=bat_id
        )


        pitcher_matrix = compute_pitcher_matrix(
            clean_df=clean_df, 
            pitcher=pitcher, 
            strikes=strikes, 
            balls=balls, 
            bat_id=bat_id
        )

        # print(f'Balls: {balls}; Strikes: {strikes};')
        all_comb_prob = prob_next_state(hitter_matrix, pitcher_matrix)
        certain_comb_prob = all_comb_prob.loc[all_comb_prob['previous_play'] == prev_outcome]
        certain_comb_prob['actual_outcome'] = actual_outcome
        certain_comb_prob['pitch_id'] = pitch_id
        results_df = pd.concat([results_df, certain_comb_prob], axis=0)
    
    # write out to csv in case something goes wrong (every 500 iterations)
    if counter_i % 500 == 0: 
        results_df.to_csv(f'results_df_{counter_i}.csv')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  from ipykernel import kernelapp as app


In [40]:
import os
# get all of the csvs that have written out from process
csvs = sorted([i for i in os.listdir() if i.startswith('results_df')])

results_df = pd.DataFrame()
for csv in csvs:
    t = pd.read_csv(csv)
    results_df = pd.concat([results_df, t], axis=0)

In [41]:
results_df.head(50)

,Unnamed: 0,previous_play,ball,contact_foul,contact_in_on_base,contact_in_play_out,strike,actual_outcome,pitch_id,strikeout,walk
0,0,first_pitch,0.350211,0.122363,0.037975,0.056259,0.433193,ball,b77ac3a4-673b-4644-b7fd-dcc720bca88b565de4be-d...,NaN,NaN
1,0,ball,0.322581,0.149194,0.060484,0.104839,0.362903,strike,b77ac3a4-673b-4644-b7fd-dcc720bca88b565de4be-d...,NaN,NaN
2,2,strike,0.344444,0.188889,0.066667,0.100000,0.300000,strike,b77ac3a4-673b-4644-b7fd-dcc720bca88b565de4be-d...,NaN,NaN
3,2,strike,0.348837,0.255814,0.116279,0.104651,NaN,contact_in_on_base,b77ac3a4-673b-4644-b7fd-dcc720bca88b565de4be-d...,0.174419,NaN
4,0,first_pitch,0.368347,0.107843,0.036415,0.079832,0.407563,strike,b7b37ed4-5752-4a4f-a581-d7312ed598aa565de4be-d...,NaN,NaN
5,1,strike,0.384880,0.230241,0.044674,0.120275,0.219931,ball,b7b37ed4-5752-4a4f-a581-d7312ed598aa565de4be-d...,NaN,NaN
6,0,ball,0.312057,0.234043,0.078014,0.127660,0.248227,contact_in_on_base,b7b37ed4-5752-4a4f-a581-d7312ed598aa565de4be-d...,NaN,NaN
7,0,first_pitch,0.370044,0.104626,0.048458,0.089207,0.387665,strike,b7b712f2-35a6-4802-8289-4f14af92d13c565de4be-d...,NaN,NaN
8,1,strike,0.474432,0.167614,0.056818,0.102273,0.198864,ball,b7b712f2-35a6-4802-8289-4f14af92d13c565de4be-d...,NaN,NaN
9,0,ball,0.328431,0.210784,0.053922,0.186275,0.220588,ball,b7b712f2-35a6-4802-8289-4f14af92d13c565de4be-d...,NaN,NaN


### Simulate for a couple random at bats to show how process words

In [45]:
# choose 20000 random at-bats
r_bat_id = random.randint(0, len(clean_df['bat_id'].unique()))
n = 5
test_bat_ids = clean_df['bat_id'].unique()[r_bat_id:r_bat_id+n]

In [47]:
results_df = pd.DataFrame()
for counter_i, test_bat_id in enumerate(test_bat_ids): 
    bat_df = clean_df.loc[clean_df['bat_id'] == test_bat_id]
    
    # get the name of the hitter and pitcher
    hitter, pitcher = bat_df['hitter_full_name'].unique().item(), bat_df['pitcher_full_name'].unique().item()
    print(f'The hitter is: {hitter}')
    print(f'The pitcher is: {pitcher}')
    total_events = sorted(bat_df['event_num'].tolist())

    for event in total_events: 
        # get the count at that event in time
        count_df = bat_df.loc[bat_df['event_num'] == event]
        pitch_id = count_df['pitch_id'].item()
        prev_outcome = count_df['prev_assigned_outcome'].item()
        actual_outcome = count_df['assigned_outcome'].item()
        balls, strikes = count_df['startingBalls'].item(), count_df['startingStrikes'].item()
        print(f'The count is {balls}-{strikes}')
        
        # pull the transition matrix for all times this count occured in the dataset
        # remove this certain at bat for both hitter and pitcher
        hitter_matrix = compute_hitter_matrix(
            clean_df=clean_df, 
            hitter=hitter, 
            strikes=strikes, 
            balls=balls, 
            bat_id=bat_id
        )
        display('Hitter Matrix ', hitter_matrix)

        pitcher_matrix = compute_pitcher_matrix(
            clean_df=clean_df, 
            pitcher=pitcher, 
            strikes=strikes, 
            balls=balls, 
            bat_id=bat_id
        )
          
        display('Pitcher Matrix ', pitcher_matrix)

        all_comb_prob = prob_next_state(hitter_matrix, pitcher_matrix)
        certain_comb_prob = all_comb_prob.loc[all_comb_prob['previous_play'] == prev_outcome]
        certain_comb_prob['actual_outcome'] = actual_outcome
        display('Events Probibility', certain_comb_prob)
        certain_comb_prob['pitch_id'] = pitch_id
        results_df = pd.concat([results_df, certain_comb_prob], axis=0)
    
    print('-' * 100)
    

The hitter is: Brian_McCann
The pitcher is: Brad_Ziegler
The count is 0-0


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  from ipykernel import kernelapp as app


'Hitter Matrix '

,previous_play,ball,contact_foul,contact_in_on_base,contact_in_play_out,strike,total
0,first_pitch,0.43908,0.091954,0.016092,0.062069,0.390805,435


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  from ipykernel import kernelapp as app


...

'Pitcher Matrix '

,previous_play,ball,contact_foul,contact_in_on_base,contact_in_play_out,strike,total
0,ball,0.365854,0.243902,0.097561,0.024390,0.268293,41
1,contact_foul,0.400000,0.100000,0.100000,0.100000,0.300000,10
2,strike,0.333333,0.047619,0.190476,0.190476,0.238095,21


'Events Probibility'

,previous_play,ball,contact_foul,contact_in_on_base,contact_in_play_out,strike,actual_outcome
1,contact_foul,0.361702,0.170213,0.021277,0.12766,0.319149,contact_in_play_out


----------------------------------------------------------------------------------------------------
